Attempting to read in the instrument data

In [14]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd

In [ ]:
# set file path to netcdf files
PATH = Path('/gws/nopw/j04/iecdt/computer-vision-data/cloudnet-vertical-profile-data/')

In [ ]:
#use xarray to read all of the netcdf files, file format .nc

data = xr.open_mfdataset(PATH.glob('*.nc'), combine='nested', join='left', concat_dim='time')

print(data)


<xarray.Dataset> Size: 445MB
Dimensions:                 (time: 60326, height: 459)
Coordinates:
  * height                  (height) float32 2kB 114.5 144.5 ... 1.384e+04
  * time                    (time) datetime64[ns] 483kB 2023-08-07T00:00:15.0...
Data variables:
    target_classification   (time, height) float64 222MB dask.array<chunksize=(2874, 459), meta=np.ndarray>
    detection_status        (time, height) float64 222MB dask.array<chunksize=(2874, 459), meta=np.ndarray>
    cloud_base_height_amsl  (time) float32 241kB dask.array<chunksize=(2874,), meta=np.ndarray>
    cloud_top_height_amsl   (time) float32 241kB dask.array<chunksize=(2874,), meta=np.ndarray>
    cloud_base_height_agl   (time) float32 241kB dask.array<chunksize=(2874,), meta=np.ndarray>
    cloud_top_height_agl    (time) float32 241kB dask.array<chunksize=(2874,), meta=np.ndarray>
    altitude                (time) float32 241kB 85.0 85.0 85.0 ... 85.0 85.0
    latitude                (time) float32 241kB 51.1

# Goals of this section #

The netCDF data contains an array target_classification that we will use as our GT. This array
includes 10 classes describing various atmospheric targets for each timestep.

Instrument Data Processing:

• Group the 10 original classes into two categories:

– No hydrometeors (clear sky): None, Aerosols & insects, Insects, Aerosols.

– Hydrometeors (cloud present): Melting & droplets, Melting ice, Ice & droplets, Ice, Drizzle &
droplets, Drizzle or rain, Droplets.


• For each timestamp, process the vertical profile to form a 459 × 2 vector. Here, each of the 459 discrete
height levels (ranging approximately from 100 m to 14 km) is assigned a pair of class probabilities
indicating the presence (hydrometeors) or absence (clear sky) of clouds

In [ ]:
# inspect data
data.target_classification

<xarray.DataArray 'target_classification' (time: 60326, height: 459)> Size: 222MB
dask.array<concatenate, shape=(60326, 459), dtype=float64, chunksize=(2880, 459), chunktype=numpy.ndarray>
Coordinates:
  * height   (height) float32 2kB 114.5 144.5 174.5 ... 1.381e+04 1.384e+04
  * time     (time) datetime64[ns] 483kB 2023-08-07T00:00:15.000000782 ... 20...
Attributes:
    units:       1
    long_name:   Target classification
    comment:     \nThis variable provides the main atmospheric target classif...
    definition:  \nValue 0: Clear sky.\nValue 1: Cloud liquid droplets only.\...

In [ ]:
# assign whether a target classification is a hydrometeor or not

# if data.target_classification == 0 OR > 7, then it is 'No Hydrometeors' (assign value 0). Else, it is 'Hydrometeors' (assign value 1). Store values in new array in data called Hydrometeors
data['Hydrometeors'] = xr.where((data.target_classification == 0) | (data.target_classification > 7), 0, 1)

# view values of hydrometer data to check if it worked
data['Hydrometeors'].load()

<xarray.DataArray 'Hydrometeors' (time: 60326, height: 459)> Size: 222MB
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(60326, 459))
Coordinates:
  * height   (height) float32 2kB 114.5 144.5 174.5 ... 1.381e+04 1.384e+04
  * time     (time) datetime64[ns] 483kB 2023-08-07T00:00:15.000000782 ... 20...

In [ ]:
# view values for target classification to compare to the above hydrometeors to check if they have been classified correctly
data.target_classification.load()

<xarray.DataArray 'target_classification' (time: 60326, height: 459)> Size: 222MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [8., 8., 8., ..., 0., 0., 0.],
       [8., 8., 8., ..., 0., 0., 0.],
       ...,
       [8., 8., 8., ..., 0., 0., 0.],
       [8., 8., 8., ..., 0., 0., 0.],
       [8., 8., 8., ..., 0., 0., 0.]], shape=(60326, 459))
Coordinates:
  * height   (height) float32 2kB 114.5 144.5 174.5 ... 1.381e+04 1.384e+04
  * time     (time) datetime64[ns] 483kB 2023-08-07T00:00:15.000000782 ... 20...
Attributes:
    units:       1
    long_name:   Target classification
    comment:     \nThis variable provides the main atmospheric target classif...
    definition:  \nValue 0: Clear sky.\nValue 1: Cloud liquid droplets only.\...

In [122]:
#Task is... For each timestamp, process the vertical profile to form a 459 × 2 vector. Here, each of the 459 discrete
#height levels (ranging approximately from 100 m to 14 km) is assigned a pair of class probabilities
#indicating the presence (hydrometeors) or absence (clear sky) of clouds

# subset the data to only include the Time, Height and Hydrometeors columns
data_subset = data[['time', 'height', 'Hydrometeors']]
data_subset.load()


<xarray.Dataset> Size: 222MB
Dimensions:       (time: 60326, height: 459)
Coordinates:
  * time          (time) datetime64[ns] 483kB 2023-08-07T00:00:15.000000782 ....
  * height        (height) float32 2kB 114.5 144.5 174.5 ... 1.381e+04 1.384e+04
Data variables:
    Hydrometeors  (time, height) int64 222MB 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
Attributes: (12/15)
    Conventions:                  CF-1.8
    cloudnetpy_version:           1.65.8
    file_uuid:                    b02973a1-c044-492d-827c-d30c02a454ac
    cloudnet_file_type:           classification
    title:                        Classification products from Chilbolton
    source_file_uuids:            09bf048d-ae74-4db3-87c4-abfde32ed17a
    ...                           ...
    year:                         2023
    source:                       RAL Space Copernicus\nVaisala CL51\nRPG-Rad...
    history:                      2024-09-06 18:02:51 +00:00 - classification...
    references:                   https://doi.org/10.21105/joss.02123
    cloudnet_processing_version:  2.42.8
    pid:                          https://hdl.handle.net/21.12132/1.b02973a1c...

In [ ]:
# set file path for images from camera A
A_DPATH = Path('/gws/nopw/j04/iecdt/computer-vision-data/cam_a/rectified_imgs')

In [ ]:
# read in images from camera A, and convert the times to datetime format
times = [file.stem for file in list(A_DPATH.glob('*.png'))]
times_datetime = pd.to_datetime(times, unit='s')

In [ ]:
# align the times from the images to the times in the data
tol = pd.Timedelta('5s')
aligned_times = data_subset.sel(time=times_datetime, method='nearest')
times_within_tol = (aligned_times.time - times_datetime) <= tol
data_aligned = aligned_times.sel(time=times_within_tol, method='nearest')

We are only using data within 5 seconds of the groun truth measurements. This means we lose approx. 1 day of measurements, which we think is trhe last day, but we still have data for over 13,000 measuermements

In [ ]:
# save the aligned times, with their respective heights and hydrometeor values, to a netCDF file
data_aligned.to_netcdf(PATH / '../../JERMIT_the_frog/hydrometeors_time_aligned_classes.nc')